# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
# read the cities.csv to create dataframe, previous cells are not needed to be executed as long as cities.csv was created
cities_data = pd.read_csv("output_data/cities.csv", encoding="utf-8")
cities_data.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,kaeo,-35.10,173.78,54.55,82,89,3.56,NZ,1602824526
1,1,maningrida,-12.07,134.27,86.32,58,65,13.35,AU,1602824526
2,2,borogontsy,62.67,131.17,22.78,91,45,5.97,RU,1602824526
3,3,saint-philippe,-21.36,55.77,80.01,57,26,5.82,RE,1602824527
4,4,busselton,-33.65,115.33,70.00,33,38,4.97,AU,1602824527


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
### Humidity Heatmap
gmaps.configure(api_key = g_key)
fig=gmaps.figure()

figure_layout = {'width' : '1000px', 'height' : '600px', 'border' : '1px solid black', 'padding' : '1px' }
locations = cities_data[['Lat','Lng']]
weights = cities_data['Humidity']
center_coordinates = (30.3333,31.2333)
fig = gmaps.figure(center = center_coordinates, map_type = "SATELLITE", zoom_level = 1.8, layout=figure_layout)



heatmap_layer = gmaps.heatmap_layer(locations,weights = weights, dissipating=False, max_intensity = 100, point_radius = 3)

fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', padding='1px', width='1000px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [4]:
clean_cities_data = cities_data.loc[(cities_data["Max Temp"] > 70) &
                                          (cities_data["Max Temp"] < 80) &
                                          (cities_data["Wind Speed"] < 10) &
                                          (cities_data["Cloudiness"] == 0)
                                          , :]
clean_cities_data = clean_cities_data.dropna(how='any')
clean_cities_data.reset_index(inplace=True)
del clean_cities_data['index']

# Display to output
clean_cities_data.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,20,san quintin,30.48,-115.95,71.64,60,0,3.36,MX,1602824530
1,33,tucupita,9.06,-62.05,73.81,95,0,2.68,VE,1602824533
2,261,kutum,14.20,24.67,74.39,20,0,8.90,SD,1602824588
3,284,hwange,-18.37,26.50,74.30,49,0,5.50,ZW,1602824593
4,299,brumado,-14.20,-41.67,72.70,64,0,5.08,BR,1602824596


In [5]:
hotel_df = clean_cities_data

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df["Hotel Name"] = ""
hotel_df["Hotel Address"] = ""
hotel_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name,Hotel Address
0,20,san quintin,30.48,-115.95,71.64,60,0,3.36,MX,1602824530,,
1,33,tucupita,9.06,-62.05,73.81,95,0,2.68,VE,1602824533,,
2,261,kutum,14.20,24.67,74.39,20,0,8.90,SD,1602824588,,
3,284,hwange,-18.37,26.50,74.30,49,0,5.50,ZW,1602824593,,
4,299,brumado,-14.20,-41.67,72.70,64,0,5.08,BR,1602824596,,


In [7]:
#Set parameters to search for hotels with 5000 meters.
target_radius = 5000
target_type = 'lodging'

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

for i in hotel_df.index:
    target_coordinates = hotel_df['Lat'][i].astype(str) + "," + hotel_df['Lng'][i].astype(str)



    params = {
      "location": target_coordinates,
      "radius": target_radius,
      "type": target_type,
      "key": g_key
    }


    response = requests.get(base_url, params=params)
    locations = response.json()

    # since some data may be missing we incorporate a try-except to skip any that are missing a data point.
    try:
        hotel_df['Hotel Name'][i] = locations["results"][0]["name"]
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

Missing field/result... skipping.


In [8]:
hotel_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name,Hotel Address
0,20,san quintin,30.48,-115.95,71.64,60,0,3.36,MX,1602824530,Old Mill Hotel,
1,33,tucupita,9.06,-62.05,73.81,95,0,2.68,VE,1602824533,Hotel Tucupita Su​ite C.A.,
2,261,kutum,14.20,24.67,74.39,20,0,8.90,SD,1602824588,Dibbo areaمنطقة دبو,
3,284,hwange,-18.37,26.50,74.30,49,0,5.50,ZW,1602824593,Hwange Colliery Guest House,
4,299,brumado,-14.20,-41.67,72.70,64,0,5.08,BR,1602824596,Hotel São Lázaro,
5,370,abu kamal,34.45,40.92,70.59,51,0,8.75,SY,1602824612,محطة التلفاز,
6,397,xai-xai,-25.05,33.64,70.57,69,0,5.21,MZ,1602824617,Platinum Hotel,
7,406,cupertino,37.32,-122.03,77.00,32,0,1.01,US,1602824619,Courtyard by Marriott San Jose Cupertino,
8,414,prainha,-1.80,-53.48,77.13,77,0,3.89,BR,1602824620,Hotel Brasília,
9,430,jumla,29.27,82.18,71.29,26,0,4.90,NP,1602824583,Hotel Sundar Rara Karnali,


In [9]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [10]:
hotel_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name,Hotel Address
0,20,san quintin,30.48,-115.95,71.64,60,0,3.36,MX,1602824530,Old Mill Hotel,
1,33,tucupita,9.06,-62.05,73.81,95,0,2.68,VE,1602824533,Hotel Tucupita Su​ite C.A.,
2,261,kutum,14.20,24.67,74.39,20,0,8.90,SD,1602824588,Dibbo areaمنطقة دبو,
3,284,hwange,-18.37,26.50,74.30,49,0,5.50,ZW,1602824593,Hwange Colliery Guest House,
4,299,brumado,-14.20,-41.67,72.70,64,0,5.08,BR,1602824596,Hotel São Lázaro,
5,370,abu kamal,34.45,40.92,70.59,51,0,8.75,SY,1602824612,محطة التلفاز,
6,397,xai-xai,-25.05,33.64,70.57,69,0,5.21,MZ,1602824617,Platinum Hotel,
7,406,cupertino,37.32,-122.03,77.00,32,0,1.01,US,1602824619,Courtyard by Marriott San Jose Cupertino,
8,414,prainha,-1.80,-53.48,77.13,77,0,3.89,BR,1602824620,Hotel Brasília,
9,430,jumla,29.27,82.18,71.29,26,0,4.90,NP,1602824583,Hotel Sundar Rara Karnali,


In [11]:

# add marker layer on top of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', padding='1px', width='1000px'))